## Four processes write a file in parallel

In the exercise you will fill out the blank spaces. 

1. Each process should write its rank (as one character) ten times to the offsets = `rank + i * size`, where `i=0..9`. 

2. Each process uses the default view. 

### Tip

When checking if your file is correctly written, you should:

- use `ls -l` to look at the number of bytes and it should be 40 bytes

- use `cat my_file` to look at the content of your file and the expected result is `0123012301230123012301230123012301230123`

- use `rm my_file` to remove the file before running the program again because it will rewrite the file

In [ ]:
?MPI::MPI_File_open

In [ ]:
?MPI::MPI_File_write_at

In [ ]:
?MPI::MPI_File_close

***
#### C skeleton

In [ ]:
#include <mpi.h>

In [ ]:
%%executable  a.x -- -lmpi

int rank, size, i;

MPI_File fh;
MPI_Status status;
__________ offset;

char buf; 

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &size);

MPI_File_open(MPI_COMM_WORLD, "my_file", 
              MPI_MODE_____ | MPI_MODE_____,
              MPI_INFO_NULL, &fh);

for (i = 0; i < 10; i++) {
    buf = '0' + (char)rank;
    offset = _________;
    MPI_File_write_at(fh, offset, &buf, ___, ________, &status);
}

MPI_File_close(&fh);

printf ("PE%d\n", rank);

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file

***
#### Python skeleton

In [ ]:
%%file writefile.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

fh = MPI.File.Open(comm, "my_file", MPI.MODE______|MPI.MODE______)

for i in range(0,10):
    buffer = np.empty(1, dtype='i')
    buffer[:] = rank
    offset = ______________
    fh.Write_at(offset, buffer)

fh.Close()

print("PE%d" % rank)

In [ ]:
!mpirun -np 4 python writefile.py

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file

***
#### Fortran skeleton

In [ ]:
%%file writefile.f90
program writefile
use mpi

integer ( kind = 4 ) error
integer :: rank, size, i

integer :: fh
integer :: status(MPI_STATUS_SIZE)
integer(_____________) :: offset

character :: buf

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

call MPI_File_open(MPI_COMM_WORLD, 'my_file', &
                   & IOR(MPI_MODE_______, MPI_MODE_______), &
                   & MPI_INFO_NULL, fh, error)

do i = 1, 10
    buf = CHAR( ICHAR('0') + rank ) 
    offset = __________
    call MPI_File_write_at(fh, offset, buf, ___, __________, &
                           & status, error)
end do
    
call MPI_File_close(fh, error)

print *, 'PE=', rank
    
call MPI_Finalize(error)
end

In [ ]:
!mpif90 writefile.f90 && mpirun -np 4 a.out

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file

***
### You can compare with our solution:

***
#### C solution

In [ ]:
%%executable  a.x -- -lmpi

int rank, size, i;

MPI_File fh;
MPI_Status status;
MPI_Offset offset;

char buf; 

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &size);

MPI_File_open(MPI_COMM_WORLD, "my_file", 
              MPI_MODE_RDWR | MPI_MODE_CREATE,
              MPI_INFO_NULL, &fh);

for (i = 0; i < 10; i++) {
    buf = '0' + (char)rank;
    offset = rank + size*i;
    MPI_File_write_at(fh, offset, &buf, 1, MPI_CHAR, &status);
}

MPI_File_close(&fh);

printf ("PE%d\n", rank);

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file

***
#### Python solution

In [ ]:
%%file writefile.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

fh = MPI.File.Open(comm, "my_file", MPI.MODE_RDWR|MPI.MODE_CREATE)

for i in range(0,10):
    buffer = np.empty(1, dtype='i')
    buffer[:] = rank
    offset = rank + size*i
    fh.Write_at(offset, buffer)

fh.Close()

print("PE%d" % rank)

In [ ]:
!mpirun -np 4 python writefile.py

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file

***
#### Fortran solution

In [ ]:
%%file writefile.f90
program writefile
use mpi

integer ( kind = 4 ) error
integer :: rank, size, i

integer :: fh
integer :: status(MPI_STATUS_SIZE)
integer( kind = MPI_OFFSET_KIND) :: offset

character :: buf

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

call MPI_File_open(MPI_COMM_WORLD, 'my_file', &
                   & IOR(MPI_MODE_RDWR, MPI_MODE_CREATE), &
                   & MPI_INFO_NULL, fh, error)

do i = 1, 10
    buf = CHAR( ICHAR('0') + rank ) 
    offset = rank + size*(i-1) 
    call MPI_File_write_at(fh, offset, buf, 1, MPI_CHARACTER, &
                           & status, error)
end do
    
call MPI_File_close(fh, error)

print *, 'PE=', rank
    
call MPI_Finalize(error)
end

In [ ]:
!mpif90 writefile.f90 && mpirun -np 4 a.out

In [ ]:
!ls -l

In [ ]:
!cat my_file

In [ ]:
!rm my_file